In [ ]:
from google.colab import drive
drive.mount('/content/drive')

FOLDERNAME = "/Colab Notebooks/최종"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


전처리 파트

In [ ]:
text = '''
파이썬은 데이터를 분석하는 과정에서 목적에 따라, 데이터를 처리하기 위한 매우 편리하고 다양한 기능을 제공합니다.
그중에서도 문자열 자르기(슬라이싱, Slicing)은 데이터의 형태나 도메인과 관계없이 핵심적인 역할을 수행하죠.
사용자는 문자열 슬라이싱을 통해 문자열의 특정 부분을 효율적으로 추출하거나, 조작할 수 있으며, 이는 데이터 분석, 웹 애플리케이션 개발, 자동화 스크립트 등 다양한 영역에서 필수적인 기술로 쓰입니다.
파이썬을 사용해 문자열의 일부분을 잘라내는 기본적인 사용법, 그리고 구체적인 작동 원리를 이해하는 것은 더욱 강력하고 유연한 코드를 작성하는 데 있어 결정적인 차이를 만듭니다.
이번 글에서는 파이썬의 문자열 슬라이싱의 기본 문법부터, C언어로 구현된 CPython의 내부 로직까지 깊게 살펴보겠습니다.
문자열 슬라이싱 사용 방법 우선 파이썬 문자열 슬라이싱은 콜론(:)을 사용하여, 문자열의 일부분을 선택하는 방식으로 이루어집니다.
기본적인 문법은 오브젝트[시작:끝:증감] 형태를 가지며, 각 인덱스(요소)는 슬라이스의 범위를 정의하는 데 중요한 역할을 합니다.
시작(start) 인덱스는 슬라이스가 시작되는 위치를 나타냅니다. 주의할 점으로, 이 위치에 해당하는 문자는 슬라이스 결과물에 포함됩니다.
만약 시작 인덱스가 생략되면 파이썬은 0을 값으로 사용, 즉, 문자열의 처음부터 슬라이스를 시작합니다.
이어서 끝(end) 인덱스는 슬라이스가 끝나는 위치를 나타냅니다. 시작 인덱스와의 차이점으로 이 위치에 해당하는 문자는 슬라이스 결과에 포함되지 않는다는 것입니다.
끝 인덱스가 주어지지 않는다면, 슬라이스는 문자열의 마지막 문자까지 포함합니다.
마지막으로 스텝(step) 값은 슬라이스 과정에서 문자를 건너뛸 간격을 지정합니다.
이 값은 선택 사항이며, 입력되지 않는다면 1을 기본값으로 사용하여 모든 문자를 순차적으로 선택합니다.
이어서 양수 및 음수 인덱스를 활용한 슬라이싱 예시를 보겠습니다.
양수 인덱스: 문자열의 처음부터 인덱싱하며, 0부터 시작합니다.
예를 들어, 문자열 s = “yozm.wishket.com”에서 s[5:12]는 인덱스 5('w')부터 11('t')까지의 부분 문자열 "wishket"을 반환합니다.
음수 인덱스: 문자열의 끝에서부터 인덱싱하며, -1부터 시작합니다.
예를 들어, 동일한 문자열 s에서 s[-6:-1]는 뒤에서 6번째 문자('e')부터 뒤에서 2번째 문자('o')까지의 부분 문자열 "et.co"를 반환합니다.
스텝을 사용하면 더욱 다양한 방식으로 문자열을 슬라이스할 수 있습니다.
양수 스텝: s[2:8:2]는 인덱스 2부터 8까지 2개 간격으로 문자를 추출하여 "z.i"를 반환합니다.
음수 스텝: s[6:1:-2]는 인덱스 6부터 1까지 역순으로 2개 간격으로 문자를 추출하여 "i.z"를 반환합니다.
특히, s[::-1]를 사용하면 텍스트 전체를 역순으로 슬라이스하여 "moc.tekhsiw.mzoy"를 얻을 수 있습니다.
문자열 슬라이싱은 실제 다양한 방식으로 활용될 수 있으며, 몇 가지 구체적인 예시는 다음과 같습니다.
특정 패턴 추출: 파일 경로에서 파일명이나 확장자를 추출하거나, URL에서 특정 부분을 분리하는 데 유용합니다.
예를 들어, 파일 경로 "C:/Users/Public/Documents/report.pdf"에서 파일명 "report"를 추출하기 위해 슬라이싱을 활용할 수 있습니다.
문자열 일부 수정 (새로운 문자열 생성): 파이썬 문자열은 불변(immutable)하므로, 슬라이싱을 통해 기존 문자열의 일부를 기반으로 새로운 문자열을 생성합니다.
예를 들어, 문자열 "Hello"의 첫 글자를 "J"로 바꾸려면, "J" + "Hello"[1:]와 같이 슬라이싱과 문자열 연결을 사용해야만 합니다.
문자열 역순 만들기: 텍스트 분석이나 데이터 처리 시 문자열을 역순으로 정렬해야 하는 경우가 있습니다.
앞서 소개한 [::-1] 슬라이스를 사용하면 간편하게 문자열을 뒤집을 수 있습니다.
불변 문자열 객체
불변이란 원래의 오브젝트의 내용을 변경할 수 없음을 의미합니다.
만약 s = ‘www.wishket.com’ 이라는 코드를 실행하면 ‘yozm.wishket.com’에서 s의 내용이 변경되고, 문제가 발생하지 않기에 내용이 변경되는 것으로 오해할 수 있는데요.
이는 메모리에 ‘www.wishket.com’이라는 문자열 오브젝트를 새롭게 만들고, 이 주소를 s에 새롭게 할당한 것입니다.
그렇기에 문자열의 내부를 변경하면 의도한 대로 불변을 변경하는 과정에서의 오류를 확인할 수 있습니다.
이 불변의 개념이 문자열 슬라이싱에서 중요한 이유는, 슬라이싱 또한 불변인 원본 문자열을 변경하지 않으면서, 문자열의 일부를 사용하여 새로운 문자열을 만드는 작업이기 때문입니다.
하지만 파이썬, 엄밀히는 CPython에서는 가능하면 메모리 복사를 피하고, 불필요한 객체 생성을 줄이기 위한 여러 최적화 전략들이 코드에 녹아있습니다.
이제 실제로 CPython이 이 슬라이싱을 처리하는 과정을 내부 코드와 함께 알아보겠습니다.
파이썬 텍스트 객체의 슬라이싱 연산은 객체의 __getitem__ 메소드에 slice 객체를 전달하여 이루어집니다.
즉, 앞서 본 s[5:12]와 아래의 slice 객체를 사용하는 코드는 동일한 역할을 합니다.
CPython에서는 unicode_subscript라는 함수를 통해 이 동작을 구현합니다.
이 함수는 인덱싱과 슬라이싱을 모두 처리하는데, 전달된 item이 정수 인덱스라면 개별 문자를 반환하고 slice 객체라면 하위 문자열을 반환하도록 되어 있습니다.
(cpython 코드 원본 참고)
이어지는 코드는 슬라이싱하는 상황(slice 객체를 사용)의 핵심 로직 중 일부로, 먼저 슬라이스 범위에 해당하는 길이 slicelength를 계산한 다음, 이 값에 따라 세 갈래의 처리를 합니다.
1. 빈 문자열 슬라이싱: 만약 slicelength가 0 이하라면 즉, 슬라이싱 결과가 빈 문자열이라면 _Py_RETURN_UNICODE_EMPTY()를 호출합니다.
이 함수는 이름에서 의미하듯 비어있는 unicode를 반환하는 함수인데, 전역적으로 미리 정의된 빈 문자열 객체를 반환합니다.
즉, 임의의 문자열 s에 대해 s[1:1]처럼 정의할 수 있는 빈 문자열 “”는 CPython에 미리 하나만 생성해 두고, 모든 곳에서 참조하는 싱글톤(singleton)이기 때문에, 불필요하게 매번 새로운 객체를 만들지 않습니다.
2. 전체 문자열 슬라이싱: start == 0, step == 1 그리고 원본 문자열의 길이와 slicelength가 같아 전체 문자열을 슬라이싱 하는 경우, 새로운 객체를 만들지 않고 원본 문자열 객체를 그대로 반환합니다.
이 방법 덕분에 아래와 같이 문자열을 같은 값으로 새롭게 정의하는 것은 다른 오브젝트로 계산되지만, 전체 문자열을 슬라이싱하는 정의 방식은 같은 오브젝트로 계산됩니다.
이를 통해 불필요한 메모리 복사를 아낄 수 있습니다.
3. 부분 문자열 슬라이싱 (새 객체 생성): 앞서 다룬 두 가지 외의 일반적인 슬라이싱을 하는 경우로, 슬라이싱한 내용을 담은 새로운 문자열 객체를 생성합니다.
여기서도 한가지 눈여겨 볼 부분은 코드가 step == 1인 경우와 step != 1인 경우로 나뉘는 것입니다.
step == 1인 경우: PyUnicode_Substring이라는 함수를 통해 원본 문자열의 start부터 slicelenth만큼을 읽어들여 새로운 오브젝트를 할당하고 해당 메모리에 문자열 데이터를 복사합니다.
step != 1인 경우: 먼저 slicelength만큼의 버퍼 영역을 메모리 할당한 뒤 원본 문자열에서 해당 인덱스의 문자들을 하나씩 복사합니다.
이어 버퍼로부터 문자열 객체를 생성하고, 버퍼를 해제합니다.
다시 정리하면, 미리 “상수”처럼 정의된 빈 문자열을 반환하거나, 전체 문자열을 그대로 슬라이싱하여 원본 객체를 반환하는 경우가 아니라면 슬라이싱은 항상 새로운 문자열 객체를 생성하고, 내용 문자를 복사하여 채웁니다.
이는 o(n) 시간 복잡도를 갖습니다.
(시간 복잡도에 대해서는아티클을 참고하세요.)
슬라이싱 결과 객체의 메모리 관리, 최적화
위에서 살펴본 구현으로부터, 슬라이싱 결과 문자열은 원본과 분리된 별도의 객체임을 알 수 있습니다.
따라서 전체 슬라이스의 예외를 빼면 원본 문자열 객체는 슬라이싱 후에도 참조 카운트가 변하지 않으며, 새로 만들어진 하위 문자열 객체의 수명이 독자적으로 관리됩니다.
이러한 설계에는 여러 가지 이유와 부가적인 내부 최적화가 존재합니다.
부분 슬라이스의 메모리 복사: 불변 객체인 문자열은 이론적으로는 뷰(view) 형태로 구현할 수도 있습니다.
예컨대 C나 Java의 일부 구현처럼, 원본 문자열 버퍼를 공유하면서 시작 오프셋과 길이만 따로 관리하는 방식도 생각해 볼 수 있습니다.
그러나 CPython은 이런 방식을 택하지 않았습니다.
왜냐하면 작은 부분 문자열이 큰 원본을 참조할 경우, 부분 문자열이 살아있는 한 거대한 원본 메모리를 해제하지 못해 메모리 누수를 유발할 수 있기 때문입니다​.
(참고)
CPython의 가비지 컬렉션은 기본적으로 참조 카운트에 기반하므로, 부분 문자열이 원본을 참조하면 원본 객체는 참조 카운트가 남아 메모리가 유지됩니다.
따라서 작은 조각을 쓰기 위해 거대한 문자열 전체를 메모리에 붙잡아 두는 상황이 발생할 수 있습니다​.
이러한 이유로 CPython은 안전하고 단순한 복사를 통한 구현을 선택했습니다.
빈 문자열과 단일 문자 캐싱: CPython은 특정 문자열들을 전역적으로 선언, 미리 캐싱하여 매번 새로 할당하지 않도록 최적화합니다.
대표적인 것이 앞서 살펴본 빈 문자열 "" 그리고 추가로 자주 쓰이는 한 글자 문자열들입니다.
앞서 설명했듯, 빈 문자열은 unicode_empty라는 전역 싱글톤 객체로 관리되며 슬라이싱 결과가 빈 문자열이면 이 객체를 반환합니다​.
또한 라틴-1 범위 (U+0000 ~ U+00FF)의 단일 문자 문자열에 대해서는 미리 256개를 만들어 unicode_latin1 배열에 캐싱해 사용합니다.
예를 들어, 슬라이싱 결과가 "A"나 "5"처럼 한 글자인 경우, 그리고 그 문자의 코드 포인트가 0~255 사이라면 CPython은 새로운 객체를 만들지 않고 이미 준비된 해당 문자를 가리키는 문자열 객체를 반환합니다.
따라서 "a"[0:1], "a"[::] 등으로 'a'를 얻으면 항상 동일한 내부 객체를 참조하게 됩니다.
반면 U+0100(256) 이상의 문자에 대해서는 기본 설정상 캐싱 되지 않아 같은 문자라도 새로운 객체가 생성됩니다.
기타 메모리 관리 (프리리스트): CPython은 작은 Unicode 객체 생성을 빠르게 하기 위해 프리리스트(freelist)도 사용합니다.
일정 크기 이하(예: 길이 1~8)의 Unicode 객체들은 해제 시 완전히 메모리를 반환하지 않고 내부 free 리스트에 보관했다가, 새로운 문자열 생성 시 재사용하는 최적화가 있습니다​.
이 방법은 슬라이싱뿐만 아니라 모든 작은 문자열 생성에 적용되며, 메모리 할당/해제의 오버헤드를 줄여줍니다.
다만 이 부분은 CPython의 내부 구현 최적화이며, 외부에서 직접적으로 확인하기는 어렵습니다.
예제로 보는 내부 처리 흐름 위 내용을 바탕으로, 실제 파이썬 코드 예제가 내부에서 어떻게 처리되는지 다시 한번 따라가 보겠습니다.
예시로는 아래의 코드를 사용합니다.
이 코드에 대한 CPython 내부 처리 과정은 다음과 같습니다.
sub1 = s[0:]: 이 슬라이스는 문자열 전체(‘Python Substring Example’)를 반환하므로, unicode_subscript 함수에서 두 번째 조건에 부합합니다.
따라서 CPython은 s 객체 자체를 반환합니다.
결과적으로 sub1 is s는 두 객체가 같은 객체이므로 True가 되고, id(sub1)과 id(s)도 동일합니다.
또한 새로운 메모리 할당이 일어나지 않으므로, 메모리 사용량 면에서도 이 연산은 저렴합니다​(참조 카운트만 증가).
sub2 = s[7:16]: 부분 문자열 "Substring"을 추출하는 슬라이스입니다.
이 경우 step=1이지만 전체 길이가 아니므로, unicode_subscript는 새로운 Unicode 객체를 생성하게 됩니다.
내부적으로 PyUnicode_FromUnicode(s->str + 7, 9) (9글자 길이)가 호출되고, 원본 s의 데이터 중 7번 인덱스부터 9개의 문자를 메모리 복사하여 새 버퍼에 채웁니다.
이 버퍼로 PyUnicodeObject 구조체가 할당되어 sub2를 가리키게 됩니다​.
이제 sub2는 s와 다른 객체이며(sub2 is s는 False), id(sub2)도 다릅니다.
그러나 sub2의 내용은 "Substring"으로 원본의 해당 부분과 동일합니다.
원본 s와 sub2는 메모리를 공유하지 않으므로, 설령 이후에 s가 참조 회수가 없어 가비지 컬렉션 되어도 sub2는 독립적으로 내용을 유지합니다.
sub3 = s[::2]: 이 슬라이스는 처음부터 끝까지 step=2로 (0, 2 , 4, ...) 문자를 취합니다.
구현상 step != 1이므로, CPython은 slicelength를 계산한 후 그 길이만큼 버퍼를 할당하고, 원본의 해당 인덱스 문자들을 하나씩 복사합니다​.
이 예에서 s의 짝수 인덱스 문자를 모아 "Pto usrn xml" 같은 결과를 얻게 될 것입니다.
이 역시 새로운 객체 sub3를 생성하며, 메모리 복사가 발생합니다.
다만 step=2라 연속되지 않은 메모리 접근이지만, 여전히 복사 비용은 결과 문자열 길이에 비례합니다.
마치며
이렇게 파이썬 문자열 슬라이싱의 기본적인 문법부터 시작하여, 인덱싱 방식, 파라미터의 역할, 문자열 불변성의 개념, 그리고 CPython 내부 구현까지 깊이 있게 살펴보았는데요.
문자열 슬라이싱은 파이썬에서 텍스트 데이터를 효율적으로 처리하기 위한 강력한 도구이며, 그 작동 원리를 정확히 이해하는 것은 사용자가 더욱 효과적이고 유연한 코드를 작성하는 데 필수적입니다.
기본 문법과 다양한 활용법을 숙지하고, 인덱싱 방식과 파라미터의 역할을 명확히 이해하며, 문자열의 불변성이라는 핵심 개념을 기억한다면, 파이썬을 이용한 문자열 조작 능력을 한 단계 더 향상시킬 수 있을 겁니다.
더 나아가 CPython 소스 코드 탐색을 통해, 슬라이싱의 내부 작동 방식을 이해한다면 파이썬 언어 자체에 대한 깊이 있는 통찰력을 얻을 수 있죠.
앞으로도 파이썬 문서, 튜토리얼, 그리고 다양한 문제 해결을 통해 문자열 슬라이싱을 포함한 프로그래밍 능력을 더욱 발전시킬 수 있길 기대합니다.
'''
title = '''
한 번에 이해하는 파이썬 '문자열 자르기(Slicing)'
'''


In [ ]:
!git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git
%cd Mecab-ko-for-Google-Colab/
!bash install_mecab-ko_on_colab_light_220429.sh

Cloning into 'Mecab-ko-for-Google-Colab'...
remote: Enumerating objects: 138, done.
remote: Counting objects: 100% (47/47), done.
remote: Compressing objects: 100% (38/38), done.
remote: Total 138 (delta 26), reused 22 (delta 8), pack-reused 91 (from 1)
Receiving objects: 100% (138/138), 1.72 MiB | 28.79 MiB/s, done.
Resolving deltas: 100% (65/65), done.
/content/Mecab-ko-for-Google-Colab/Mecab-ko-for-Google-Colab/Mecab-ko-for-Google-Colab
Installing konlpy.....
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2025-05-27 08:17:41--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket.org (bitbucket.org)... 104.192.142.25, 104.192.142.26, 104.192.142.24, ...
Connecting to bitbucket.org (bitbucket.org)|104.192.142.25|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://bbuseruploads.s3.amazonaws.com/eunjeon/mecab-ko/downloads/m

In [ ]:
import sys
sys.path.append("/content/drive/MyDrive/Colab Notebooks/최종")
from preprocessing import normalize_parentheses, extract_nouns_numbers, extract_parts
import pandas as pd

In [ ]:
import pandas as pd
df_general = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Terminolgy_score/Data/genraltest.csv", encoding="utf-8-sig")
df_general

,id,all_text
0,0,저는 개인적으로 복사가 터졌으면 좋겠어요. 자 여러분들 피티알 2일차 새롭게 알게 ...
1,1,안녕하세요. 여러분들의 디아 친구 르넵니다. 21일 새벽 디아블로 포 시즌 포에 대...
2,2,여러분 패치 노트가 드디어 공개됐고요. 단 영문판을 구글 번역기로 돌린 버전입니다....
3,3,항저우 아시안 게임 이제 또 아주 괄목할 만한 그런 또 어떤 성적을 내 주셨는데 그...
4,4,이번에 이런 기사가 났습니다. 이 스포츠 토토 2024년 엘시케이 도입 목표 추진....
...,...,...
3029,3029,요즘 취미로 운동 하나 새로 시작했잖아. 탁구를 시작하는데 지금 한 2달 정도 됐잖...
3030,3030,당신은 좋아하는 음식이 뭐야? 돼지고기가 제일 맛있는데 어제는 맛이 없었어. 나는 ...
3031,3031,나는 당신이랑 결혼하고 나서 나의 취미가 없어진 거 같아. 취미 아니 그러니까 등산...
3032,3032,우리 결혼한 지도 10년이 넘어서 이번에 10주년 결혼 여행을 가자고 했잖아. 근데...


In [ ]:

data = {
    'id': ['user_text'],
    'domain' : 'ai',
    "content": [text],
    "terms": [extract_nouns_numbers(text)],
    "PART": [extract_parts(text)]
}

df = pd.DataFrame(data)
df

df_general["terms"] = df_general['all_text'].apply(extract_nouns_numbers)


In [ ]:
# 예시: df['terms'] 에서 각 리스트 내 단어 중, 길이 <= max_len 인 것만 남기기
max_len = 30

# 1) df['terms'] 안의 이중 리스트(혹은 다중 중첩)를 미리 평탄화(flatten)하는 대신,
#    바로 각 행의 terms 리스트 내에서 필터링해 주는 방법
df['terms'] = df['terms'].apply(
    lambda term_list: [t for t in term_list if isinstance(t, str) and len(t) <= max_len]
)


In [ ]:
df['PART'].to_list()

[[['그중에서도 문자열 자르기', '슬라이싱, Slicing'],
  ['시작', 'start'],
  ['이어서 끝', 'end'],
  ['마지막으로 스텝', 'step'],
  ['5', "'w'"],
  ['11', "'t'"],
  ['문자', "'e'"],
  ['문자', "'o'"],
  ['파이썬 문자열은 불변', 'immutable'],
  ['코드는 슬라이싱하는 상황', 'slice 객체를 사용'],
  ['곳에서 참조하는 싱글톤', 'singleton'],
  ['문자열은 이론적으로는 뷰', 'view'],
  ['0100', '256'],
  ['하기 위해 프리리스트', 'freelist'],
  ['일정 크기 이하', '예: 길이 1~8'],
  ['슬라이스는 문자열 전체', '‘Python Substring Example’'],
  ['id', 'sub1'],
  ['PyUnicode_FromUnicode', 's->str + 7, 9'],
  ['다른 객체이며', 'sub2 is s는 False'],
  ['id', 'sub2']]]

Rule BASE

In [ ]:
import re
from itertools import chain, repeat
from collections import Counter
general_stopwords = set(chain.from_iterable(df_general["terms"]))

PRIMARY = ['paren_eng','abbr','code_id','func_call','list_item']
def has_primary(term, text):
    return (
        re.search(rf'\b{re.escape(term)}\b\s*\(\s*[A-Za-z]', text)      # paren_eng
     or re.fullmatch(r'[A-Z]{2,}(?:-[A-Z0-9]+)*', term)                # abbr
     or re.fullmatch(r'[a-z].*_[A-Za-z0-9_]+', term)                   # snake_case
     or re.fullmatch(r'[A-Z][a-z0-9]+(?:[A-Z][a-z0-9]+)+', term)       # CamelCase
     or re.search(rf'\b{re.escape(term)}\b\s*(?=[,\/]|등\b)', text)    # list_item
     or re.fullmatch(r'[가-힣]+\( *[A-Za-z0-9 _\-:]+ *\)', term)

    )

WEIGHTS = {
    'paren_eng' : 4,   # 1) 괄호 내 영문 병기
    'abbr'      : 3,   # 2) 영문 약어/대문자
    'code_id'   : 2,   # 3) snake_case 또는 CamelCase
    'func_call' : 2,   # 4) 함수/메서드 호출 형태
    'list_item' : 2,   # 5) 나열 항목
    'special'   : 2,   # 6) 특수문자 포함
    'length'    : 1,   # 7) 길이 ≥ 3 또는 단어 수 ≥ 2
    'freq'      : 1,   # 8) 본문 내 2회 이상 반복
    'early'     : 1,   # 9) 문서 앞 20% 이내 첫 등장
    'single_eng'  : 1,
    'paren_only_eng'  : 0
}

def score_term(term: str, text: str) -> int:
    """
    term: 후보 용어
    text: 전체 본문
    return: 적용된 Rule 가중치 합산 점수
    """
    score = 0

    # 1) 괄호 내 영문 병기: 한글 용어 뒤에 (영문…) 이 나오는 경우
    if re.search(r'([가-힣]{2,})\s*\(\s*([A-Za-z][A-Za-z0-9\- ]*[A-Za-z])\s*\)', text):
        score += WEIGHTS['paren_eng']


    # 2) 영문 약어/대문자: 전부 대문자 2자 이상
    if re.fullmatch(r'[A-Z]{2,}(?:-[A-Z0-9]+)*', term):
        score += WEIGHTS['abbr']

    # 3) 코드식 식별자: snake_case 또는 CamelCase
    #    snake_case 예: unicode_subscript, __getitem__
    #    CamelCase 예: CPython, PyUnicode_Substring
    if re.fullmatch(r'[a-z][A-Za-z0-9_]*_[A-Za-z0-9_]+', term) or \
       re.fullmatch(r'[A-Z][a-z0-9]+(?:[A-Z][a-z0-9]+)+', term):
        score += WEIGHTS['code_id']

    # 4) 함수/메서드 호출 형태: name()
    if re.fullmatch(r'[A-Za-z_][A-Za-z0-9_]*\(\)', term):
        score += WEIGHTS['func_call']

    # 5) 나열 항목: 쉼표, 슬래시, “등” 앞에 오는 경우
    if re.search(rf'\b{re.escape(term)}\b\s*(?=[,\/]|등\b)', text):
        score += WEIGHTS['list_item']

    # 6) 특수문자 포함: _ - ( ) [ ] .
    if re.search(r'[_\-\(\)\[\]\.]', term):
        score += WEIGHTS['special']

    # 7) 길이 ≥ 3 또는 단어 수 ≥ 2
    if len(term) >= 3 or len(term.split()) >= 2:
        score += WEIGHTS['length']

    # 8) 본문 내 반복 출현 (2회 이상)
    if text.count(term) >= 2:
        score += WEIGHTS['freq']

    # 9) 문서 앞 20% 이내 첫 등장
    idx = text.find(term)
    if 0 <= idx <= len(text) * 0.2:
        score += WEIGHTS['early']
    # 10) 단일 영문 단어: 오직 A–Z, a–z 로만 이루어진 경우
    if re.fullmatch(r'[A-Za-z]+', term):
        score += WEIGHTS['single_eng']

    if re.search(rf'\(\s*{re.escape(term)}\s*\)', text):
        score += WEIGHTS['paren_only_eng']

    return score

In [ ]:
def filter_specialized_terms(text, raw_terms,  threshold=5):
    flat = list(dict.fromkeys(chain.from_iterable(raw_terms)))
    # 2) 형태소 기반 명사구(n>=2) 후보를 우선 추가

        # 2) 괄호 안의 영문(숫자·_, -, :, 공백 포함)만 뽑아 추가
    for eng in re.findall(r'\(\s*([A-Za-z0-9_\- :]+?)\s*\)', text):
        if eng not in flat:
            flat.append(eng)

    # 3) 한글+괄호영문(용어(설명)) 복합어를 “용어(설명)” 형태로 추가
    for kor, eng in re.findall(
        r'([가-힣]+)\s*\(\s*([A-Za-z0-9_\- :]+?)\s*\)',
        text
    ):
        comp = f"{kor}({eng})"
        if comp not in flat:
            flat.append(comp)
    results = []
    print(flat)
    for term in flat:
        if not has_primary(term, text):
            continue
                          # 1차 필터링: 강력한 Rule 하나도 없으면 스킵
        if '(' in term:
           kor = term.split('(', 1)[0]
           if kor in general_stopwords:
                continue
        else:
          if term in general_stopwords:
              continue
        print(term)

        sc = score_term(term, text)
        if sc >= threshold:
            results.append((term, sc))
    return results


In [ ]:
if __name__ == "__main__":
    text
    raw_terms = df["terms"]  # 이중리스트 구조
    general_stopwords: set[str]

    # 1) 초기 전문용어 단일어 추출
    singles_RB = filter_specialized_terms(text, raw_terms, threshold=4)

['파이썬', '데이터', '분석', '과정', '목적', '처리', '편리', '기능', '제공', '문자열', '(슬라이 싱 , Slicing)', '형태', '도메인', '핵심', '역할', '수행', '사용자', '슬라이', '특정', '부분', '효율', '추출', '조작', '애플리케이션', '개발', '자동', '스크립트', '다양', '영역', '필수', '기술', '사용', '일부분', '기본', '사용법', '구체', '작동', '원리', '이해', '코드', '작성', '결정', '차이', '문법', '언어', '구현', 'CPython', '내부', '로직', '방법', '콜론', '(:)', '선택', '방식', '오브젝트', '시작', '증감', '인덱스', '(요소)', '슬라이스', '범위', '정의', '중요', '(start)', '위치', '주의', '해당', '문자', '결과물', '포함', '생략', '처음', '(end)', '차이점', '결과', '마지막', '스텝', '(step)', '간격', '지정', '사항', '입력', '순차', '양수', '음수', '활용', '예시', '인덱싱', 'yozm', 'wishket', 'com', '12', '불변', '(immutable)', '기존', '일부', '기반', '생성', 'Hello', '글자', '이싱', '연결', '역순', '텍스트', '정렬', '경우', '소개', '객체', '원래', '내용', '변경', '의미', 'www', '실행', '문제', '발생', '오해', '메모리', '주소', '할당', '의도', '대로', '오류', '확인', '개념', '이유', '원본', '작업', '때문', '가능', '복사', '필요', '최적화', '전략', '연산', 'getitem', '메소드', 'slice', '전달', '아래', 'unicode', 'subscript', '함수', '동작', 'item', '정수', '개별', '반환', '하위', 

In [ ]:
singles_RB

[('스크립트', 8),
 ('불변', 5),
 ('RETURN', 9),
 ('UNICODE', 9),
 ('EMPTY', 9),
 ('싱글톤', 6),
 ('PyUnicode', 9),
 ('id', 6),
 ('FromUnicode', 8),
 ('PyUnicodeObject', 8),
 ('콜론(:)', 8),
 ('끝(end)', 8),
 ('불변(immutable)', 7),
 ('싱글톤(singleton)', 7),
 ('뷰(view)', 7),
 ('프리리스트(freelist)', 7)]

In [ ]:
df_RB = pd.DataFrame(singles_RB, columns=['term', 'score'])
df_RB

,term,score
0,스크립트,8
1,불변,5
2,RETURN,9
3,UNICODE,9
4,EMPTY,9
5,싱글톤,6
6,PyUnicode,9
7,id,6
8,FromUnicode,8
9,PyUnicodeObject,8


TF_IDF_SUB

In [ ]:
from TFIDF_SUB import TFIDF_SUB

In [ ]:
terms = extract_nouns_numbers(text)
mean_df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Terminolgy_score/Data/mean_tf_idf_sub.csv', encoding='utf-8-sig')
mean_tfidf = dict(zip(mean_df['word'], mean_df['mean_tf_idf']))
# 3) 교집합만 남기고 평균 TF–IDF값으로 정렬
candidates = [(t, mean_tfidf[t]) for t in set(terms) if t in mean_tfidf]
top20_tfidf = sorted(candidates, key=lambda x: x[1], reverse=True)[:20]

print("Top specialized terms (by avg TF–IDF):")
for term, score in top20_tfidf:
    print(f"{term}: {score:.4f}")

Top specialized terms (by avg TF–IDF):
slice: 0.0076
item: 0.0055
메소드: 0.0051
증감: 0.0042
empty: 0.0042
불변: 0.0040
버퍼: 0.0038
스크립트: 0.0038
id: 0.0037
콜론: 0.0036
free: 0.0031
인덱싱: 0.0031
오버헤드: 0.0026
카운트: 0.0026
싱글톤: 0.0025
최적화: 0.0024
step: 0.0023
도메인: 0.0023
yozm: 0.0022
com: 0.0022


In [ ]:
import pandas as pd
from TFIDF_SUB import TFIDF_SUB

# 1) 기존 balanced corpus 로딩
df_extract = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Terminolgy_score/Data/yozm_extract_04.csv", converters={'terms': lambda x: x.split()})
# terms 컬럼이 리스트 형태로 저장돼 있다면, 적절한 converter 사용

# 2) text 전처리 + user row 생성
terms = extract_nouns_numbers(text)
df_user = pd.DataFrame({
    'id': ['user_text'],
    'domain': ['ai'],            # IT로 표시
    'content': [text],
    'terms': [terms],
    'PART': [[]],                # PART가 필요 없으면 빈 리스트
})
df_all = pd.concat([df_extract, df_user], ignore_index=True)

# 3) TFIDF_SUB 파이프라인 실행
tfidf = TFIDF_SUB(df_all, ast=False)
tfidf.TF()
tfidf.DF()
tfidf.TF_IDF()

# 4) user_text 행에서 per-doc TF–IDF 꺼내기
user_row = tfidf.df.loc[tfidf.df['id']=='user_text', 'tf_idf'].iloc[0]
top20_tfidf2 = sorted(user_row.items(), key=lambda x: x[1], reverse=True)[:20]

print("Top specialized terms (per-doc TF–IDF):")
for term, score in top20_tfidf2:
    print(f"{term}: {score:.4f}")


Counting Term Frequency


100%|██████████| 6072/6072 [00:29<00:00, 204.65it/s]


Normalizing Term Frequency


100%|██████████| 6072/6072 [00:00<00:00, 18661.50it/s]


TF Calculation Finished
Creating List of Words


...

DF Calculation Finished
Calculating TF*IDF for IT Domain


100%|██████████| 3038/3038 [00:02<00:00, 1290.64it/s]


Caclulating Average TF*IDF


  0%|          | 0/3038 [00:00<?, ?it/s]/content/drive/MyDrive/Colab Notebooks/최종/TFIDF_SUB.py:126: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  tf_idf_dict = row[tf_idf_idx]
100%|██████████| 3038/3038 [00:01<00:00, 2932.44it/s]

All Finished
Top specialized terms (per-doc TF–IDF):
: 0.0895
s: 0.0523
슬라이: 0.0477
문자열: 0.0477
문자: 0.0389
n: 0.0302
이싱: 0.0273
Py: 0.0209
객체: 0.0173
CPython: 0.0161
슬라이스: 0.0129
sub: 0.0123
인덱스: 0.0097
메모리: 0.0092
반환: 0.0085
원본: 0.0084
slice: 0.0084
Unicode: 0.0066
불변: 0.0063
step: 0.0062


In [ ]:
df20_tfidf  = pd.DataFrame(top20_tfidf,  columns=['term', 'score'])
df20_tfidf2 = pd.DataFrame(top20_tfidf2, columns=['term', 'score'])

# 합치기 (term을 기준으로 중복 제거, top20 우선)
df_tfidf = pd.concat([df20_tfidf, df20_tfidf2]).drop_duplicates(subset='term', keep='first').reset_index(drop=True)

# 점수를 소수점 5자리로 반올림
df_tfidf['score'] = df_tfidf['score'].round(5)

In [ ]:
def is_single_letter_english(term):
    return bool(re.fullmatch(r'[A-Za-z]', term))

def is_substring_of_other(term: str, terms_list: list[str]) -> bool:
    """
    같은 df 안의 terms_list(예: df_tfidf['term'].tolist())에서
    term이 다른 term의 부분 문자열로 들어가 있으면 True.
    """
    for other in terms_list:
        # 1) 자기 자신은 스킵
        # 2) term이 빈 문자열이거나
        # 3) term이 other 안에 포함되어 있으면 True
        if other != term and term and term in other:
            return True
    return False
terms = df_tfidf['term'].tolist()
filtered = []
for term, score in zip(df_tfidf['term'], df_tfidf['score']):
    if term == '':
        continue
    if is_single_letter_english(term):
        continue
    #  동일 df의 terms 목록으로 비교
    if is_substring_of_other(term, terms):
        continue
    filtered.append((term, score))
filtered2 = [(term, score) for term, score in filtered if score > 0.0022]
df_tfidf = pd.DataFrame(filtered2,  columns=['term', 'score'])
df_tfidf

,term,score
0,slice,0.00758
1,item,0.00554
2,메소드,0.00507
3,증감,0.00421
4,empty,0.00416
5,불변,0.00400
6,버퍼,0.00385
7,스크립트,0.00381
8,id,0.00373
9,콜론,0.00356


강화학습

In [ ]:
import numpy as np
term = np.load("/content/drive/MyDrive/Colab Notebooks/최종/term_np.npy")
import pandas as pd
extract_df = df_extract
import ast
general_term = np.load("/content/drive/MyDrive/Colab Notebooks/최종/general_term_np.npy")


In [ ]:
def reward_fn(word):
    if word in term:
        return 1.0
    else:
        return -1.0

tokens = df.loc[0, "terms"]


In [ ]:
import gym
import torch
import torch.nn as nn
from torch.distributions import Categorical
import spacy

# ---------- 1. 문서 및 명사구 추출(일반 용어 제외) ----------
nlp = spacy.load("en_core_web_sm")
tokens = [x for x in tokens if x not in general_term]
# ---------- 2. 보상 함수 ----------
manually_approved_terms = set()

# ---------- 3. 환경 ----------
class TermExtractionEnv(gym.Env):
    def __init__(self, document, reward_fn):
        self.document = document
        self.reward_fn = reward_fn
        self.current_index = 0
        self.selected_terms = []

    def reset(self):
        self.current_index = 0
        self.selected_terms = []
        return self._get_state()

    def _get_state(self):
        return self.document[self.current_index]

    def step(self, action):
        current_word = self.document[self.current_index]
        reward = 0

        if action == 1:  # select
            self.selected_terms.append(current_word)
            reward = self.reward_fn(current_word)
        else:
            reward = 0

        self.current_index += 1
        done = self.current_index >= len(self.document)

        return self._get_state() if not done else None, reward, done, {}

# ---------- 4. 정책 모델 ----------
class SimplePolicy(nn.Module):
    def __init__(self, vocab, embed_dim):
        super().__init__()
        self.vocab = vocab
        self.word_to_idx = {word: i for i, word in enumerate(vocab)}
        self.embedding = nn.Embedding(len(vocab), embed_dim)
        self.policy_net = nn.Sequential(
            nn.Linear(embed_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 2),
        )

    def forward(self, word):
        idx = torch.tensor([self.word_to_idx[word]])
        embedded = self.embedding(idx)
        return self.policy_net(embedded.squeeze(0))

# ---------- 5. 보조 함수 ----------
def compute_discounted_rewards(rewards, gamma=0.99):
    R = 0
    returns = []
    for r in reversed(rewards):
        R = r + gamma * R
        returns.insert(0, R)
    return returns

# ---------- 6. 학습 ----------
vocab = list(set(tokens))
env = TermExtractionEnv(tokens, reward_fn)
policy = SimplePolicy(vocab=list(set(tokens)), embed_dim=32)
optimizer = torch.optim.Adam(policy.parameters(), lr=0.01)

from tqdm import trange
num_episodes = 3

for episode in trange(num_episodes):
    state = env.reset()
    log_probs = []
    rewards = []
    done = False

    while not done:
        logits = policy(state)
        dist = Categorical(logits=logits)
        action = dist.sample()
        log_probs.append(dist.log_prob(action))

        state, reward, done, _ = env.step(action.item())
        rewards.append(reward)

    returns = compute_discounted_rewards(rewards)
    loss = -sum(lp * R for lp, R in zip(log_probs, returns))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

100%|██████████| 3/3 [00:00<00:00,  6.50it/s]


In [ ]:
threshold = 0.7

# (2) 단어별로 “term_prob”을 구해서, 조건(>=threshold)을 만족하는 것만 모읍니다.
records = []
for word in tokens:
    if word not in policy.word_to_idx:
        continue
    with torch.no_grad():
        logits = policy(word)
        probs  = F.softmax(logits, dim=-1)
        term_prob = probs[1].item()      # action=1(전문용어) 확률
    if term_prob >= threshold:
        records.append({
            'term': word,
            'probability': term_prob
        })

# (3) DataFrame으로 변환 & 중복 제거(원래 순서 유지)
df_reinforce = pd.DataFrame(records)
df_reinforce = df_reinforce.drop_duplicates(subset='term', keep='first').reset_index(drop=True)
df_reinforce = df_reinforce[~df_reinforce['term'].str.contains(r'[()]')].reset_index(drop=True)

print(df_reinforce)

          term  probability
0          com     0.705924
1           불변     0.738237
2        Hello     0.701325
3          메소드     0.731441
4        slice     0.757355
5  slicelength     0.701417
6       RETURN     0.726863


In [ ]:
df_reinforce

,term,probability
0,EMPTY,0.549258


점수 총합 및 결과

In [ ]:
#추출한 리스트에서 중복되어서 존재하면 전문용어
#동일하지 않더라도 PMI를 사용해서 유사도 높으면, 같은 걸로 치고 둘 중 긴거로 전문용어

In [ ]:
df_all = pd.concat([df_RB, df_tfidf,df_reinforce], ignore_index=True)
df_all['term'] = df_all['term'].str.strip().str.lower()
# 3) 그룹핑: 짧은 term이 긴 term에 포함되면 긴 term으로 합산 (한글·영어 모두 적용)
terms = df_all['term'].tolist()
mapping = {}
for term in terms:
    parents = [other for other in terms if other != term and term in other]
    if parents:
        # 가장 길이가 긴 부모 term 선택
        mapping[term] = max(parents, key=len)
    else:
        mapping[term] = term

df_all['canonical'] = df_all['term'].map(mapping)

# 4) canonical 기준으로 score 합산
df_final = df_all.groupby('canonical', as_index=False)['score'].sum()
df_final = df_final.sort_values('score', ascending=False).round({'score':5}).reset_index(drop=True)

# 결과 표시
df_final

,canonical,score
0,pyunicodeobject,26.00664
1,싱글톤(singleton),13.00246
2,불변(immutable),12.00400
3,empty,9.00416
4,return,9.00000
5,스크립트,8.00381
6,콜론(:),8.00356
7,끝(end),8.00000
8,fromunicode,8.00000
9,프리리스트(freelist),7.00312


In [ ]:
import re
import math
import pandas as pd

# 1) 슬라이딩 윈도우
def sliding_windows(text: str, window_size: int) -> list[str]:
    return [text[i : i + window_size] for i in range(len(text) - window_size + 1)]

# 2) 빈도/공기빈도
def count_occurrences(text: str, sub: str) -> int:
    return len(re.findall(re.escape(sub), text))

def count_cooccurrence(wins: list[str], a: str, b: str) -> int:
    return sum(1 for w in wins if a in w and b in w)

# 3) PMI 계산
def compute_pmi(text: str, a: str, b: str, window_size: int) -> float:
    wins   = sliding_windows(text, window_size)
    N      = len(wins)
    cnt_a  = count_occurrences(text, a)
    cnt_b  = count_occurrences(text, b)
    cnt_ab = count_cooccurrence(wins, a, b)

    p_a  = cnt_a  / N if N else 0
    p_b  = cnt_b  / N if N else 0
    p_ab = cnt_ab / N if N else 0
    if p_a == 0 or p_b == 0 or p_ab == 0:
        return float('-inf')
    return math.log(p_ab / (p_a * p_b), 2)

# 4) “단어 내부”에 있는 경우만 prefix 뽑기
def extract_prefixes(text: str, target: str, max_len: int = 5) -> list[str]:
    prefixes = set()
    for m in re.finditer(re.escape(target), text):
        # (1) 맨 앞이거나, 앞 글자가 공백이면 standalone → 건너뛰기
        if m.start() == 0 or text[m.start() - 1].isspace():
            continue
        # (2) 실제 단어 내부이므로, L=2~max_len 글자 앞부분을 후보로
        for L in range(2, max_len + 1):
            start = m.start() - L
            if start < 0:
                continue
            cand = text[start : m.start()]
            # 맨뒤에 공백 포함된 조각도 skip
            if cand and not cand[-1].isspace():
                prefixes.add(cand)
    return list(prefixes)

# 5) canonical 교정
def correct_canonical(df: pd.DataFrame,
                      text: str,
                      window_size: int = 20,
                      pmi_threshold: float = 1.0,
                      max_prefix_len: int = 5) -> pd.DataFrame:

    fragments = df["canonical"].unique().tolist()
    mapping = {}

    for frag in fragments:
        # (A) 내부 occurrence에서만 prefixes 뽑기
        prefixes = extract_prefixes(text, frag, max_len=max_prefix_len)

        # (B) 가장 높은 PMI인 접두 선택
        best, best_pmi = None, float("-inf")
        for pre in prefixes:
            pmi = compute_pmi(text, pre, frag, window_size)
            if pmi > best_pmi:
                best_pmi, best = pmi, pre

        # (C) threshold 만족 시에만 교정 맵핑
        if best and best_pmi >= pmi_threshold:
            mapping[frag] = best + frag

    # (D) df_final에 맵핑 적용
    df_fixed = df.copy()
    df_fixed["canonical"] = df_fixed["canonical"].apply(lambda x: mapping.get(x, x))
    return df_fixed


In [ ]:
df_fixed = correct_canonical(df_final, text,
                            window_size=5,
                            pmi_threshold=2,
                            max_prefix_len=2)

print("교정 전:\n", df_final, "\n")
print("교정 후:\n", df_fixed)

교정 전:
           canonical     score
0   pyunicodeobject  26.00664
1    싱글톤(singleton)  13.00246
2     불변(immutable)  12.00400
3             empty   9.00416
4            return   9.00000
5              스크립트   8.00381
6             콜론(:)   8.00356
7            끝(end)   8.00000
8       fromunicode   8.00000
9   프리리스트(freelist)   7.00312
10          뷰(view)   7.00000
11               id   6.00373
12              문자열   0.04767
13             슬라이싱   0.02728
14               객체   0.01728
15          cpython   0.01612
16             슬라이스   0.01288
17            며(sub   0.01232
18              인덱스   0.00969
19              메모리   0.00924
20               반환   0.00850
21               원본   0.00840
22            slice   0.00758
23             item   0.00554
24              메소드   0.00507
25             끝:증감   0.00421
26               버퍼   0.00385
27              인덱싱   0.00308
28             오버헤드   0.00265
29              카운트   0.00260
30              최적화   0.00243
31             step   0.00235
32 

In [ ]:
df_final = df_fixed

In [ ]:
df_fixed

,canonical,score
0,pyunicodeobject,26.00664
1,싱글톤(singleton),13.00246
2,불변(immutable),12.00400
3,empty,9.00416
4,return,9.00000
5,스크립트,8.00381
6,콜론(:),8.00356
7,끝(end),8.00000
8,fromunicode,8.00000
9,프리리스트(freelist),7.00312


from matplotlib import pyplot as plt
_df_0['index'].plot(kind='hist', bins=20, title='index')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_1['score'].plot(kind='hist', bins=20, title='score')
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
_df_2.plot(kind='scatter', x='index', y='score', s=32, alpha=.8)
plt.gca().spines[['top', 'right',]].set_visible(False)

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  xs = series['index']
  ys = series['score']
  
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_3.sort_values('index', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('index')
_ = plt.ylabel('score')

from matplotlib import pyplot as plt
import seaborn as sns
def _plot_series(series, series_name, series_index=0):
  palette = list(sns.palettes.mpl_palette('Dark2'))
  counted = (series['index']
                .value_counts()
              .reset_index(name='counts')
              .rename({'index': 'index'}, axis=1)
              .sort_values('index', ascending=True))
  xs = counted['index']
  ys = counted['counts']
  plt.plot(xs, ys, label=series_name, color=palette[series_index % len(palette)])

fig, ax = plt.subplots(figsize=(10, 5.2), layout='constrained')
df_sorted = _df_4.sort_values('index', ascending=True)
_plot_series(df_sorted, '')
sns.despine(fig=fig, ax=ax)
plt.xlabel('index')
_ = plt.ylabel('count()')

from matplotlib import pyplot as plt
_df_5['index'].plot(kind='line', figsize=(8, 4), title='index')
plt.gca().spines[['top', 'right']].set_visible(False)

from matplotlib import pyplot as plt
_df_6['score'].plot(kind='line', figsize=(8, 4), title='score')
plt.gca().spines[['top', 'right']].set_visible(False)

In [ ]:
# Min-Max Scaling: 0~10 사이로 변환
min_score = df_fixed['score'].min()
max_score = df_fixed['score'].max()

df_fixed['score'] = ((df_fixed['score'] - min_score) / (max_score - min_score)) * 10


In [ ]:
# term이 'sub'인 행 삭제 (index 기준이 아니라 내용 기준 추천)
df_fixed = df_fixed[df_fixed['canonical'] != '며(sub'].reset_index(drop=True)

In [ ]:
df_fixed

,canonical,score
0,pyunicodeobject,10.000000
1,싱글톤(singleton),4.999233
2,불변(immutable),4.615274
3,empty,3.461684
4,return,3.460084
5,스크립트,3.076998
6,콜론(:),3.076902
7,끝(end),3.075533
8,fromunicode,3.075533
9,프리리스트(freelist),2.692182


In [ ]:
top5_terms = df_final.nlargest(5, 'score')['canonical'].tolist()

In [ ]:
top5_terms

['pyunicodeobject', '싱글톤(singleton)', '불변(immutable)', 'empty', 'return']

RAG

In [ ]:
!pip install faiss-cpu

In [ ]:
from RAG import search_and_save

In [ ]:
open_key
db_csv_path ='/content/drive/MyDrive/Colab Notebooks/최종/aiDictionary_DB.csv'
TEST = top5_terms
search_and_save(TEST,openai_api_key=open_key, db_csv_path = db_csv_path,title=title)


----- pyunicodeobject 검색 시작 -----
[정의] 파이썬 내에서 유니코드 문자열을 나타내는 객체입니다.
[상세설명] 파이썬에서 문자열은 일반 문자열과 유니코드 문자열로 나눌 수 있는데, pyunicodeobject는 유니코드 문자열을 저장하고 처리하기 위한 객체입니다. 유니코드 문자열은 다양한 언어의 문자를 표현할 수 있는 문자열 형태로, 다국어 환경에서 텍스트를 다룰 때 유용하게 사용됩니다. pyunicodeobject는 문자열 처리 및 조작을 위한 다양한 메서드와 기능을 제공하여 유니코드 문자열을 쉽게 다룰 수 있도록 도와줍니다. 이를 통해 다국어 지원이 필요한 프로젝트나 코드에서 유용하게 활용될 수 있습니다.

----- 싱글톤(singleton) 검색 시작 -----
[정의] 싱글톤은 애플리케이션 전체에서 단 하나의 인스턴스만 생성되는 디자인 패턴입니다.
[상세설명] 싱글톤 패턴은 하나의 클래스가 오직 하나의 객체만을 생성하도록 보장하는 디자인 패턴입니다. 이는 전역 변수를 통해 클래스의 인스턴스를 미리 생성하고, 생성된 인스턴스를 반환하는 메서드를 제공하여 구현됩니다. 이를 통해 어디서든지 해당 클래스의 인스턴스에 접근할 수 있고, 불필요한 객체 생성을 방지하여 자원을 효율적으로 관리할 수 있습니다. 주로 설정값이나 로깅, 데이터베이스 연결 등과 같이 단 하나의 인스턴스가 필요한 경우에 활용됩니다.

----- 불변(immutable) 검색 시작 -----
[정의] 변하지 않고 고정된 상태를 유지하는 것
[상세설명] 불변(immutable)은 변경되지 않고 변하지 않는 속성을 말합니다. 예를 들어, 불변한 데이터 유형은 한 번 만들어지면 변경될 수 없는 데이터로, 새로운 값이 할당될 때마다 새로운 데이터를 생성합니다. 이는 코드의 안정성을 높이고 예상치 못한 버그를 방지하는 데 도움이 됩니다.

----- empty 검색 시작 -----
[정의] 데이터나 변수가 아무 값도 가지지 않은 상태를 의미합니다.
[상세설명] 어떤 변수나 